In [18]:
import logging
from copy import deepcopy

from langchain_community.document_loaders import TextLoader
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.storage import InMemoryStore
from langchain_classic.retrievers.parent_document_retriever import ParentDocumentRetriever
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains import RetrievalQA

In [2]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})
model = ChatOpenAI(model='gpt-5-mini', temperature=0.2)

In [3]:
loaders = [TextLoader('./data/How_to_invest_money.txt', encoding='utf-8')]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [4]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)

In [5]:
vectorstore = Chroma(collection_name='split_parents', embedding_function=embeddings)
docstore = InMemoryStore()

In [6]:
pc_retriever = ParentDocumentRetriever(
    docstore=docstore,
    vectorstore=vectorstore,
    parent_splitter=parent_splitter,
    child_splitter=child_splitter
)

pc_retriever.add_documents(docs)

In [7]:
print(len(list(docstore.yield_keys())))
print(vectorstore._collection.count())

219
1148


In [8]:
query = 'What are the types of investments?'
retrieved_docs = pc_retriever.invoke(query)
print(retrieved_docs[0].page_content)
print('--'*30)
sub_docs = vectorstore.similarity_search(query)
print(sub_docs[0].page_content)

There are five chief points to be considered in the selection of all
forms of investment. These are: (1) safety of principal and interest;
(2) rate of income; (3) convertibility into cash; (4) prospect of
appreciation in intrinsic value; (5) stability of market price.

Keeping these five general factors in mind, the present chapter will
discuss real-estate mortgages as a form of investment, both as adapted
to the requirements of private funds and of a business surplus.
------------------------------------------------------------
There are five chief points to be considered in the selection of all
forms of investment. These are: (1) safety of principal and interest;


In [9]:
logging.basicConfig()
logging.getLogger('langchain_classic.retrievers.multi_query').setLevel(logging.INFO)

In [10]:
mq_retriever = MultiQueryRetriever.from_llm(
    retriever=pc_retriever,
    llm=model
)

In [11]:
question = '주식 투자를 처음 시작하려면 어떻게 해야 하나요?'
unique_docs = mq_retriever.invoke(question)
print(f'{len(unique_docs)}개의 문서가 검색되었습니다.')

INFO:langchain_classic.retrievers.multi_query:Generated queries: ['주식 투자를 처음 시작할 때 따라야 할 단계별 가이드는 무엇인가요?', '초보자가 계좌 개설부터 종목 선정, 리스크 관리까지 주식 투자를 시작하는 구체적 방법을 알려주세요.', '주식 투자 초보자가 알아야 할 기본 개념과 추천 학습자료(책·강의) 및 초보자에게 적합한 투자 전략은 무엇인가요?']


7개의 문서가 검색되었습니다.


In [12]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=mq_retriever,
    chain_type='stuff',
    return_source_documents=True
)

In [13]:
result = qa_chain.invoke({'query':question})
print(result['result'])

INFO:langchain_classic.retrievers.multi_query:Generated queries: ['주식 투자를 처음 시작하려면 구체적으로 어떤 준비와 단계(계좌 개설, 증권사 선택, 기초 공부 등)를 거쳐야 하나요?', '주식 초보로서 소액으로 시작할 때 추천하는 투자 방법과 리스크 관리, 초보자가 흔히 저지르는 실수는 무엇인가요?', '주식 투자 기초 개념(매수·매도 원리, 호가·거래체결, 손절·익절)과 처음 읽어야 할 교재·강의·참고 자료는 무엇인가요?']


처음 주식 투자를 시작할 때는 체계적으로 준비하는 것이 중요합니다. 아래 순서대로 한 단계씩 해보세요.

1. 투자 목표와 기간 정하기  
   - 자본 보존이 목적인지(안전), 배당 수익을 원하는지(수익), 자본 이득(가격 상승)을 노리는지 등 목표를 분명히 하세요. 투자 기간(단기·중기·장기)에 따라 전략이 달라집니다.

2. 주식과 채권의 차이 이해하기  
   - 주식은 회사의 소유지분으로 배당과 가격 변동이 있고 위험과 수익이 큽니다. 채권은 빌려준 돈에 대한 채무증서로 원리금이 정해져 있어 상대적으로 안전합니다. 자신의 목표에 맞는 자산을 선택하세요.

3. 투자에 중요하게 보는 다섯 가지 요소 기억하기  
   - (1) 원금·이자(수익)의 안전성, (2) 수익률, (3) 현금화(환금성), (4) 내재가치 상승 전망, (5) 시장가격의 안정성. 이 중 무엇을 중시할지 정하고, 그에 맞는 종목·상품을 고르세요.

4. 자신의 위험 성향(리스크 허용도) 점검  
   - 손실을 얼마나 견딜 수 있는지, 투자 손실이 생겼을 때 대응 방식을 미리 생각하세요. 위험을 못 견디면 주식 비중을 낮추고 채권·예금 비중을 늘리는 게 낫습니다.

5. 기초 공부하기  
   - 회계·재무제표의 기본(매출, 이익, 부채비율 등), 산업과 기업의 경쟁력, 경제·금융 뉴스 보는 법을 익히세요. 무턱대고 “높은 수익률”만 보고 들어가면 안 됩니다 — 높은 수익은 그만큼의 위험이나 적극적 관리를 의미합니다.

6. 초보자는 분산투자부터 시작  
   - 개별주만 오래 들고 가기보다 ETF(지수형·섹터형)나 인덱스 펀드로 시장 전체에 분산 투자하는 것이 위험 관리에 유리합니다. 이후 경험 쌓이면 일부 자금을 개별주에 배분하세요.

7. 실거래 준비: 증권사 선택·계좌 개설  
   - 수수료, 거래 플랫폼, 고객지원, 모바일 앱 등을 비교해 증권사를 정하고 투자계좌를 만드세요. 소액으로 먼저 연습해 보세요(모의투자·소액 분할매수).

8. 투자 규칙 만들기·정기적 투자 습관 

In [15]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [19]:
def create_virtual_doc_chain():
    system = '당신은 고도로 숙련된 AI입니다. 질문에 맞도록 적절하게 행동하세요.'
    user = '''
    주어진 질문 "{query}"에 대해 직접적으로 답변하는 가상의 문서를 생성하세요.
    문서의 크기는 {chunk_size} 언저리여야 합니다.
    해당 답변은 추후 다시 질문하는데 쓰일 예정임을 참고하세요.'''

    prompt = ChatPromptTemplate.from_messages(
        [('system', system),
         ('human', user)]
    )

    return prompt | model | StrOutputParser()

In [20]:
def create_retrieval_chain():
    return RunnableLambda(lambda x: mq_retriever.invoke(x['virtual_doc']))

In [21]:
def create_final_response_chain():
    template = '''다음 정보와 질문을 바탕으로 답변해주세요.
    컨텍스트: {context}
    질문: {question}
    답변:
    '''
    final_prompt = ChatPromptTemplate.from_template(template)

    return final_prompt | model | StrOutputParser()